https://python.langchain.com/docs/tutorials/llm_chain/

在这个快速入门中，我们将向您展示如何使用LangChain构建一个简单的LLM应用程序。这个应用程序将会把文本从英语翻译成另一种语言。这是一个相对简单的LLM应用——它只需要一次LLM调用加上一些prompt。尽管如此，这是开始使用LangChain的一个很好的方式——仅通过一些prompt和LLM调用就能构建出许多功能！  

这个教程涉及如下内容：  
1. 启用LangSmith  
2. 使用本地LLM服务  
3. 使用模型Streaming模式对话  
4. 创建并使用prompt模板

下面这一步需要在LangSmith网站上申请LangSmith api key。

In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

执行下面这一步需要在本地搭建LLM服务：  
使用ollama下载qwen2或deepseek-r1:32b  
后台启动ollama服务： nohup ollama serve  >/dev/null 2>&1 &  

In [2]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(
    #model_name="deepseek-r1:32b",
    model_name="qwen2",
    openai_api_base="http://127.0.0.1:11434/v1",
    openai_api_key="EMPTY",
    streaming=False
)

In [ ]:
"""
首先，让我们直接使用该模型。ChatModels是LangChain Runnables的实例，这意味着它们提供了一个与之交互的标准接口。要简单地调用模型，我们可以将消息列表传递给.invoke方法。这种方法允许用户以一种标准化的方式与模型进行互动，使得调用模型变得直接且易于操作。这种方式非常适合用于测试和快速验证想法。
"""

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 22, 'total_tokens': 26, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-aec81ced-d691-43e4-96ec-210512645d3c-0', usage_metadata={'input_tokens': 22, 'output_tokens': 4, 'total_tokens': 26, 'input_token_details': {}, 'output_token_details': {}})

因为我们启用了LangSmith，我们可以看到这次运行已经被记录到LangSmith中，并且可以查看LangSmith跟踪。LangSmith跟踪报告了令牌使用情况信息、延迟、标准模型参数（如温度）以及其他信息。

通过LangSmith，开发者能够详细监控和评估他们的应用程序，不仅有助于优化性能，还能确保模型的安全性和稳定性。这个功能对于持续改进应用程序以及了解模型在实际应用中的表现非常有用。

In [8]:
"""
注意，ChatModels接收消息对象作为输入，并生成消息对象作为输出。除了文本内容外，消息对象还传达了对话角色并保存重要数据，如工具调用和令牌使用计数。
LangChain还支持通过字符串或OpenAI格式进行聊天模型输入。以下三种调用方式是等效的：
"""

#model.invoke("Hello")

#model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 20, 'total_tokens': 30, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-05a1cd4b-cc8a-4d1b-b63b-ffe7912a6eea-0', usage_metadata={'input_tokens': 20, 'output_tokens': 10, 'total_tokens': 30, 'input_token_details': {}, 'output_token_details': {}})

In [9]:
"""
由于chat models是Runnables，它们暴露了一个标准接口，该接口包括异步和流式调用模式。这使我们能够从聊天模型中streaming传输单个token。
"""

for token in model.stream(messages):
    print(token.content, end="|")

C|iao|!||

In [ ]:
"""
目前，我们是直接将消息列表传递到语言模型中。这个消息列表是从哪里来的呢？通常，它是由用户输入和应用程序逻辑组合构建的。这种应用程序逻辑通常会获取原始用户输入，并将其转换成准备传递给语言模型的消息列表。常见的转换包括添加系统消息或使用用户输入格式化模板。

提示模板是LangChain中设计来辅助进行这种转换的概念。它们接收原始用户输入，并返回准备好传递给语言模型的数据（prompt）。

让我们在这里创建一个提示模板。它将接收两个用户变量：
- language: 要翻译成的语言
- text: 要翻译的文本

注意，ChatPromptTemplate支持在单个模板中包含多个消息角色。我们将language参数格式化为系统消息，并将text格式化为用户消息。
"""

from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [11]:
"""
提示模板的输入是一个字典。下面我们试用这个提示模板，来看看它单独使用时的效果。
"""

prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [12]:
"""
我们可以看到，它返回了一个由SystemMessage和HumanMessage组成的ChatPromptValue。如果我们想要直接访问这些消息，可以这样做：
"""

prompt.to_messages()


[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [13]:
"""
最后，我们可以使用这个格式化的prompt调用聊天模型：
"""

response = model.invoke(prompt)
print(response.content)

Ciao!
